In [1]:
import pandas as pd
jd = pd.read_csv('../input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv')
resume = pd.read_csv('../input/resume-dataset/UpdatedResumeDataSet.csv')

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
import numpy as np
import pandas as pd
import transformers

In [3]:
com = []
for c in resume['Category'].unique():
    if(c in jd['title'].unique()):
        com.append(c)

In [4]:
jd1 = jd[jd['title'].isin(com)]
jd1 = jd1[['title','description']]
resume1 = resume[resume['Category'].isin(com)]
resume1 = resume1.rename(columns={'Category': 'title'})

In [5]:
print(f'resume shape : {resume1.shape} jd shape : {jd1.shape}')

In [6]:
for i in range(len(resume1)):
    s = resume1.iloc[i,1]
    s = ' '.join(w for w in s.split() if w.isalnum())
    s = s.strip()
    resume1.iloc[i,1] = s

In [7]:
resume1 = resume1[resume1['Resume'].str.split().str.len().lt(1000)]

In [8]:
for i in range(len(jd1)):
    s = jd1.iloc[i,1]
    s = ' '.join(w for w in s.split() if w.isalnum())
    s = s.strip()
    jd1.iloc[i,1] = s

In [9]:
resume1.head()

In [10]:
jd1.head()

In [11]:
max_resume_len = max([len(s.split()) for s in resume1['Resume']])
print(max_resume_len)

In [12]:
import matplotlib.pyplot as plt
l = [len(s.split()) for s in resume1['Resume']]
plt.hist(l)

In [13]:
max_jd_len = max([len(s.split()) for s in jd1['description']])
print(max_jd_len)

In [14]:
jd_count = jd1.groupby('title').count()
resume_count = resume1.groupby('title').count()
jd_count['Resume'] = resume_count['Resume']
jd_count.plot.bar()

In [15]:
len(resume1)

In [16]:
len(jd1)

In [17]:
data = []
for i in range(len(jd1)):
    for j in range(len(resume1)):
        l = []
        l.append(jd1.iloc[i,1])
        l.append(resume1.iloc[j,1])
        if jd1.iloc[i,0]==resume1.iloc[j,0]:
            l.append(0)
        else:
            l.append(1)
        data.append(l)

In [18]:
data1 = pd.DataFrame(data, columns = ['description', 'resume', 'similarity'])

In [19]:
df = data1.sample(frac=1).reset_index(drop=True)

In [20]:
y = tf.keras.utils.to_categorical(df.similarity, num_classes=2)

In [21]:
from sklearn.model_selection import train_test_split

train_df, valid_df,y_train, y_val = train_test_split(df,y, test_size=0.2)

In [22]:
train_df.groupby('similarity').count()

In [23]:
max_length = 512  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 2

In [24]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [25]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(2, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

In [28]:
train_data = BertSemanticDataGenerator(
    train_df[["description", "resume"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    valid_df[["description", "resume"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

In [29]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

In [30]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [35]:
bert_model.trainable = True
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

In [36]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

In [33]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data[0])[0]
    return 1-max(proba)

In [34]:
sentence1 = df.iloc[0,0]
sentence2 = df.iloc[0,1]
check_similarity(sentence1, sentence2)